In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/working/validation_gpt_data.pkl
/kaggle/working/train_gpt_data.pkl
/kaggle/working/finalmodel2.pth
/kaggle/working/checkpoints/last_epoch.txt
/kaggle/working/checkpoints/last_checkpoint/model.safetensors
/kaggle/working/checkpoints/last_checkpoint/random_states_0.pkl
/kaggle/working/checkpoints/last_checkpoint/optimizer.bin


In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
train_df = pd.read_pickle('/kaggle/input/embeddings/train_embeddings.pkl')
val_df = pd.read_pickle('/kaggle/input/embeddings/val_embeddings.pkl')

In [4]:
train_df

,question,answer,question_embeddings,answer_embeddings
4,i feel sleepy during office hours even after 8...,feeling sleepy during office timing is most co...,"[-0.26551718, -0.14716196, -0.31462595, 0.2012...","[-0.20625764, -0.33222866, -0.5742723, 0.17007..."
5,where is the best place to start learning to b...,the web is full of healthyrelated advice which...,"[-0.29065245, -0.10290378, -0.55747473, 0.1643...","[-0.19209254, -0.1223455, -0.5607167, 0.143770..."
7,weird rash over body i got a really itchy sens...,lots of things can cause an itchy rash thousan...,"[-0.22826323, 0.049277473, -0.46793014, 0.1466...","[-0.26471904, -0.0949292, -0.44640425, 0.16129..."
10,my brother in law accidentlty poked his hand w...,sorry to hear about this it is hard to tell wh...,"[-0.19800055, 0.009556178, -0.47398186, 0.2088...","[-0.25098494, -0.13435361, -0.3495351, 0.07525..."
15,what are the symptoms of pinkeye and is it con...,children under any suspicion of pinkeye will b...,"[-0.27357608, -0.08484824, -0.44519228, 0.1813...","[-0.18809138, -0.29502445, -0.5213447, 0.21331..."
...,...,...,...,...
47596,12 weeks pregnant due jan 20th last period apr...,hi again let us see if this helps you so your ...,"[-0.23717599, -0.0022189366, -0.46322224, 0.19...","[-0.31732213, -0.13892588, -0.45899993, 0.1522..."
47599,i am starting to turn gray but i am not ready ...,you can use rinses instead of hair dyes or per...,"[-0.23984611, -0.04564248, -0.41020805, 0.1701...","[-0.290672, -0.11531881, -0.54826593, 0.162990..."
47600,how is hypothyroidism diagnosed,your doctor will ask you about your symptoms a...,"[-0.2715611, -0.15185031, -0.34319144, 0.21706...","[-0.21644855, -0.066988036, -0.47266102, 0.200..."
47601,my teen has anxiety which is controlled by med...,often medications taken over a period of time ...,"[-0.17348967, 0.010912182, -0.50446445, 0.1478...","[-0.24016945, -0.12829559, -0.54014903, 0.2128..."


In [5]:
val_df

,question,answer,question_embeddings,answer_embeddings
0,why my glucose level drop so much after consum...,pancreatitis can cause insulin derangements in...,"[-0.22465815, -0.040735334, -0.43371361, 0.152...","[-0.2187827, -0.07795966, -0.29455113, 0.17727..."
4,does prozac show up on a drug test,not unless the drug test is specifically looki...,"[-0.19333406, -0.1531981, -0.46534637, 0.15835...","[-0.27048987, -0.09347108, -0.51278424, 0.1544..."
7,what are the risks in an endarterectomy for wo...,i do believe that you have no other option exc...,"[-0.26119143, -0.1005428, -0.45904684, 0.17318...","[-0.22919253, -0.026269164, -0.4260606, 0.1588..."
8,what does it mean if a food product is labeled...,the fda sets rules for what food manufacturers...,"[-0.26061082, -0.022312118, -0.43202183, 0.083...","[-0.19321632, -0.24611172, -0.614113, 0.121839..."
10,my daughter zones out for 520 seconds often he...,sounds like my historyepilepsy at age 14 compl...,"[-0.2818151, 0.010599099, -0.2714289, 0.151786...","[-0.23899251, 0.0012992404, -0.315292, 0.12722..."
...,...,...,...,...
11890,what are the symptoms of a cataract,the most common symptoms of a cataract are clo...,"[-0.29087853, -0.21130046, -0.4244496, 0.20425...","[-0.29276365, -0.18895769, -0.57395875, 0.1902..."
11893,my six year old cut himself with rusty nail no...,i do not think it is tetanus unless the wound ...,"[-0.1964538, -0.05507811, -0.4414535, 0.191567...","[-0.22308405, -0.026650071, -0.19944909, 0.094..."
11894,i had a epidural for pain today in my lumbar i...,hi thanks for the query many a time post epidu...,"[-0.23133133, -0.038379483, -0.28095374, 0.153...","[-0.2001123, -0.11215544, -0.32805523, 0.14762..."
11898,what to do if you cannot get a diagnosis but c...,you may need a more comprehensive cardiac work...,"[-0.24037237, 0.0064388383, -0.44193134, 0.062...","[-0.21089466, -0.03629826, -0.44637397, 0.0968..."


In [6]:
# Load question and answer embeddings from .npy files
train_question_embeddings = train_df['question_embeddings']
train_answer_embeddings = train_df['answer_embeddings']
val_question_embeddings = val_df['question_embeddings']
val_answer_embeddings = val_df['answer_embeddings']

In [7]:
# Function to L2 normalize embeddings
def normalize(embedding):
    norm = np.linalg.norm(embedding)
    return embedding / norm if norm > 0 else embedding

# Normalize each embedding
train_question_embeddings_norm = np.array([normalize(emb) for emb in train_question_embeddings])
train_answer_embeddings_norm = np.array([normalize(emb) for emb in train_answer_embeddings])
val_question_embeddings_norm = np.array([normalize(emb) for emb in val_question_embeddings])
val_answer_embeddings_norm = np.array([normalize(emb) for emb in val_answer_embeddings])

In [8]:
# Create DataFrames for the embeddings
train_gpt_data = pd.DataFrame({
    'question': train_df['question'].to_list(),
    'answer': train_df['answer'].to_list(),
    'Q_FFNN_embeds': list(train_question_embeddings_norm),
    'A_FFNN_embeds': list(train_answer_embeddings_norm)
})

val_gpt_data = pd.DataFrame({
    'question': val_df['question'].to_list(),
    'answer': val_df['answer'].to_list(),
    'Q_FFNN_embeds': list(val_question_embeddings_norm),
    'A_FFNN_embeds': list(val_answer_embeddings_norm)
})

# Save the processed training and validation data to disk
train_gpt_data.to_pickle('./train_gpt_data.pkl')
val_gpt_data.to_pickle('./validation_gpt_data.pkl')

In [9]:
# Load processed embeddings
train_gpt_data = pd.read_pickle("./train_gpt_data.pkl")
validation_gpt_data = pd.read_pickle("./validation_gpt_data.pkl")


In [10]:
train_gpt_data

,question,answer,Q_FFNN_embeds,A_FFNN_embeds
0,i feel sleepy during office hours even after 8...,feeling sleepy during office timing is most co...,"[-0.015292753, -0.008475955, -0.018121228, 0.0...","[-0.012246236, -0.019725576, -0.03409655, 0.01..."
1,where is the best place to start learning to b...,the web is full of healthyrelated advice which...,"[-0.017639488, -0.0062451563, -0.03383274, 0.0...","[-0.011490919, -0.007318672, -0.033541907, 0.0..."
2,weird rash over body i got a really itchy sens...,lots of things can cause an itchy rash thousan...,"[-0.013287391, 0.0028684824, -0.027238598, 0.0...","[-0.01591705, -0.005707911, -0.026841434, 0.00..."
3,my brother in law accidentlty poked his hand w...,sorry to hear about this it is hard to tell wh...,"[-0.0117288185, 0.0005660726, -0.028076928, 0....","[-0.014676283, -0.0078562945, -0.02043898, 0.0..."
4,what are the symptoms of pinkeye and is it con...,children under any suspicion of pinkeye will b...,"[-0.016663516, -0.005168105, -0.027116654, 0.0...","[-0.011119753, -0.017441519, -0.03082132, 0.01..."
...,...,...,...,...
23797,12 weeks pregnant due jan 20th last period apr...,hi again let us see if this helps you so your ...,"[-0.013904416, -0.00013008491, -0.027156351, 0...","[-0.018717567, -0.008194683, -0.027074574, 0.0..."
23798,i am starting to turn gray but i am not ready ...,you can use rinses instead of hair dyes or per...,"[-0.014035215, -0.0026708876, -0.024004383, 0....","[-0.017434262, -0.006916725, -0.032884527, 0.0..."
23799,how is hypothyroidism diagnosed,your doctor will ask you about your symptoms a...,"[-0.015845068, -0.008860174, -0.02002456, 0.01...","[-0.012838074, -0.0039732186, -0.02803464, 0.0..."
23800,my teen has anxiety which is controlled by med...,often medications taken over a period of time ...,"[-0.010499785, 0.0006604172, -0.03053074, 0.00...","[-0.014414711, -0.0077001625, -0.03241916, 0.0..."


In [ ]:
validation_gpt_data

In [12]:
# Convert embeddings to numpy format for FAISS
question_embeddings = np.stack(train_gpt_data['Q_FFNN_embeds'].values).astype('float32')
answer_embeddings = np.stack(train_gpt_data['A_FFNN_embeds'].values).astype('float32')

In [14]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.7 MB/s eta 0:00:00:00:0100:01


In [15]:
import faiss

res = faiss.StandardGpuResources()
answer_index = faiss.GpuIndexFlatIP(res, answer_embeddings.shape[1])  # Initialize GPU index  # Inner product (cosine similarity)
faiss.normalize_L2(answer_embeddings)
answer_index.add(answer_embeddings)

# # Initialize FAISS index for answer embeddings (cosine similarity)
# answer_index = faiss.IndexFlatIP(answer_embeddings.shape[1]) 
# faiss.normalize_L2(answer_embeddings)
# answer_index.add(answer_embeddings)

# # Later, if you need to load the index:
# cpu_index_loaded = faiss.read_index("./answer_index.faiss")

In [ ]:
# # Save FAISS index to disk (CPU version)
# cpu_index = faiss.index_gpu_to_cpu(answer_index)

# # Save the CPU index to disk
# faiss.write_index(cpu_index, "./answer_index.faiss")
# # faiss.write_index(answer_index, "./answer_index.faiss")

In [16]:
from transformers import GPT2Tokenizer

# Load tokenizer for GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2",use_fast=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [23]:
def prepare_gpt_input(question, answer, question_embedding, top_k=20, similarity_threshold=0.8):
    # Normalize and search FAISS index
    faiss.normalize_L2(np.array([question_embedding]).astype('float32'))
    distances, indices = answer_index.search(np.array([question_embedding]), top_k)
    
    # Filter based on similarity threshold
    similar_qas = [
        (train_gpt_data.iloc[idx]['question'], train_gpt_data.iloc[idx]['answer'])
        for idx, dist in zip(indices[0], distances[0])
        if dist >= similarity_threshold
    ]
    
    # Generate context string
    context_string = f"`QUESTION: {question} `ANSWER: {answer}"
    for q, a in similar_qas:
        context_string = f"`QUESTION: {q} `ANSWER: {a} " + context_string
        if len(tokenizer.encode(context_string)) >= 1024:
            break
    
    # Tokenize and truncate
    tokenized_input = tokenizer.encode(context_string, truncation=True, max_length=1024)
    return tokenized_input[-1024:]


In [17]:
print(type(answer_index))


<class 'faiss.swigfaiss.GpuIndexFlatIP'>


In [ ]:
from tqdm import tqdm
# Batch tokenization to improve efficiency
batch_size = 8  # Adjust as needed based on memory capacity
def batch_prepare_gpt_input(data, batch_size=batch_size):
    tokenized_inputs = []
    for i in tqdm(range(0, len(data), batch_size)):
        batch = data.iloc[i:i + batch_size]
        tokenized_batch = [
            prepare_gpt_input(row['question'], row['answer'], row['Q_FFNN_embeds'])
            for _, row in batch.iterrows()
        ]
        tokenized_inputs.extend(tokenized_batch)
    return tokenized_inputs

train_gpt_data['gpt_input'] = batch_prepare_gpt_input(train_gpt_data, batch_size)
validation_gpt_data['gpt_input'] = batch_prepare_gpt_input(validation_gpt_data, batch_size)

 19%|█▉        | 579/2976 [05:33<23:13,  1.72it/s]

In [2]:
import pandas as pd
import numpy as np
train_gpt_data = pd.read_pickle('./train_gpt_input_data.pkl')
validation_gpt_data = pd.read_pickle('./validation_gpt_input_data.pkl')

In [3]:
train_gpt_data

,question,answer,Q_FFNN_embeds,A_FFNN_embeds,gpt_input
0,i feel sleepy during office hours even after 8...,feeling sleepy during office timing is most co...,"[-0.015292753, -0.008475955, -0.018121228, 0.0...","[-0.012246236, -0.019725576, -0.03409655, 0.01...","[63, 35780, 2849, 25, 329, 262, 1613, 1285, 13..."
1,where is the best place to start learning to b...,the web is full of healthyrelated advice which...,"[-0.017639488, -0.0062451563, -0.03383274, 0.0...","[-0.011490919, -0.007318672, -0.033541907, 0.0...","[63, 35780, 2849, 25, 1312, 373, 1297, 1312, 7..."
2,weird rash over body i got a really itchy sens...,lots of things can cause an itchy rash thousan...,"[-0.013287391, 0.0028684824, -0.027238598, 0.0...","[-0.01591705, -0.005707911, -0.026841434, 0.00...","[63, 35780, 2849, 25, 1312, 423, 257, 340, 296..."
3,my brother in law accidentlty poked his hand w...,sorry to hear about this it is hard to tell wh...,"[-0.0117288185, 0.0005660726, -0.028076928, 0....","[-0.014676283, -0.0078562945, -0.02043898, 0.0...","[63, 35780, 2849, 25, 616, 11989, 9941, 15881,..."
4,what are the symptoms of pinkeye and is it con...,children under any suspicion of pinkeye will b...,"[-0.016663516, -0.005168105, -0.027116654, 0.0...","[-0.011119753, -0.017441519, -0.03082132, 0.01...","[63, 35780, 2849, 25, 460, 257, 1200, 423, 901..."
...,...,...,...,...,...
23797,12 weeks pregnant due jan 20th last period apr...,hi again let us see if this helps you so your ...,"[-0.013904416, -0.00013008491, -0.027156351, 0...","[-0.018717567, -0.008194683, -0.027074574, 0.0...","[63, 35780, 2849, 25, 1312, 716, 3058, 9661, 1..."
23798,i am starting to turn gray but i am not ready ...,you can use rinses instead of hair dyes or per...,"[-0.014035215, -0.0026708876, -0.024004383, 0....","[-0.017434262, -0.006916725, -0.032884527, 0.0...","[63, 35780, 2849, 25, 28107, 7888, 768, 4190, ..."
23799,how is hypothyroidism diagnosed,your doctor will ask you about your symptoms a...,"[-0.015845068, -0.008860174, -0.02002456, 0.01...","[-0.012838074, -0.0039732186, -0.02803464, 0.0...","[63, 35780, 2849, 25, 1312, 716, 257, 3261, 42..."
23800,my teen has anxiety which is controlled by med...,often medications taken over a period of time ...,"[-0.010499785, 0.0006604172, -0.03053074, 0.00...","[-0.014414711, -0.0077001625, -0.03241916, 0.0...","[63, 35780, 2849, 25, 318, 340, 1744, 284, 423..."


In [3]:
from torch.utils.data import Dataset
import torch

class GPTDataset(Dataset):
    def __init__(self, gpt_data, tokenizer, max_length=1024):
        self.gpt_data = gpt_data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.gpt_data)
    
    def __getitem__(self, idx):
        # Get the GPT input (question + answer) for the idx
        gpt_input = self.gpt_data['gpt_input'].iloc[idx]
        
        # Ensure input length is within max_length
        tokenized_input = self.tokenizer.encode(gpt_input, truncation=True, padding='max_length', max_length=self.max_length)
        
        # Calculate mask start index (handle `ANSWER:` position)
        mask_start = self.max_length - tokenized_input[::-1].index(self.tokenizer.encode("`ANSWER:")[0]) + 1
        loss_mask = [0] * mask_start + [1] * (self.max_length - mask_start)
        
        # Convert to tensors
        input_ids = torch.tensor(tokenized_input, dtype=torch.long)
        label = input_ids[1:]  # Shift labels by one position
        loss_mask = torch.tensor(loss_mask[:-1], dtype=torch.float32)  # Align mask with labels

        return {'input_ids': input_ids, 'labels': label, 'loss_mask': loss_mask}


In [4]:
import torch.nn.functional as F
    
# Define the cross entropy loss with ignore_index set to a non-vocab index, e.g., -1 for masked tokens
def masked_loss_fn(logits, labels, mask):
    active_logits = logits[mask == 1]
    active_labels = labels[mask == 1]
     # Check if there are any active elements; otherwise, set a small constant loss
    if active_logits.size(0) == 0:
        return torch.tensor(0.0, requires_grad=True).to(logits.device)
    
    return F.cross_entropy(active_logits, active_labels, ignore_index=-1)

In [6]:
!pip install evaluate rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=0791ca0cfdde3ed9ceca642760ae5b93e0c1d3aa11b6b0e840d5a55a4d5d478b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [5]:
from tqdm import tqdm
import torch
from accelerate import Accelerator
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorWithPadding
from torch.utils.data import DataLoader
import evaluate
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer  # Needed for the rouge metric

# Initialize evaluation metrics
perplexity_metric = evaluate.load("perplexity")
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")


def prepare_dataloader(train_data, val_data, tokenizer, batch_size=4):
    train_dataset = GPTDataset(train_data, tokenizer)
    validation_dataset = GPTDataset(val_data, tokenizer)
    
    data_collator = DataCollatorWithPadding(tokenizer, padding='longest')
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=data_collator, pin_memory=True)
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, collate_fn=data_collator, pin_memory=True)
    return train_dataloader, validation_dataloader

In [6]:
def evaluate_model(model, dataloader, accelerator, tokenizer):
    model.eval()
    eval_loss, correct_predictions, total_predictions = 0, 0, 0
    all_predictions, all_labels = [], []
    # max_batches = 2 
    # for i, batch in enumerate(tqdm(dataloader, desc="Evaluating", leave=False)):
    #     if i >= max_batches:
    #         break
    for batch in tqdm(dataloader, desc="Evaluating", leave=False):
        with torch.no_grad():
            context = batch['input_ids'].to(accelerator.device)
            label = batch['labels'].to(accelerator.device)
            loss_mask = batch['loss_mask'].to(accelerator.device)

            with accelerator.autocast():
                outputs = model(context)
                logits = outputs.logits[:, :label.size(1), :]
                masked_loss = torch.nan_to_num(masked_loss_fn(logits, label, loss_mask))
                eval_loss += masked_loss.item()

                predictions = torch.argmax(logits, dim=-1)
                correct_predictions += ((predictions == label) * loss_mask).sum().item()
                total_predictions += loss_mask.sum().item()

                # Decode predictions and labels in-place
                all_predictions.extend(tokenizer.batch_decode(predictions, skip_special_tokens=True))
                all_labels.extend(tokenizer.batch_decode(label, skip_special_tokens=True))


    eval_loss /= len(dataloader)
    accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0


    # Metrics computation
    bleu_score = bleu_metric.compute(predictions=all_predictions, references=[[ref] for ref in all_labels])
    rouge_score = rouge_metric.compute(predictions=all_predictions, references=[[ref] for ref in all_labels])
    perplexity = torch.exp(torch.tensor(eval_loss))
        

    return eval_loss, accuracy, bleu_score, rouge_score, perplexity

In [7]:
# File paths and constants
import os
CHECKPOINT_DIR = "./checkpoints2"
MODEL_SAVE_PATH = "./finalmodel2.pth"
EPOCHS=2
BATCH_SIZE=5
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

In [8]:
# Function to save the model, optimizer state, and epoch
def save_checkpoint(model, optimizer, accelerator, epoch):
    accelerator.save_state(os.path.join(CHECKPOINT_DIR, "last_checkpoint"))
    with open(os.path.join(CHECKPOINT_DIR, "last_epoch.txt"), "w") as f:
        f.write(str(epoch))
    print(f"Checkpoint saved for epoch {epoch + 1}")

In [9]:
# Function to load a checkpoint if available
def load_checkpoint(accelerator):
    checkpoint_path = os.path.join(CHECKPOINT_DIR, "last_checkpoint")
    if os.path.exists(checkpoint_path):
        accelerator.load_state(checkpoint_path)
        print(f"Checkpoint loaded from {checkpoint_path}")
        
        # Load saved epoch number
        with open(os.path.join(CHECKPOINT_DIR, "last_epoch.txt"), "r") as f:
            start_epoch = int(f.read().strip()) + 1
            print("epoch_loaded")
        return start_epoch
    else:
        print("No checkpoint found. Starting from scratch.")
        return 0

In [10]:
# Main training function
def training_pipeline(train_data, val_data, epochs=3, batch_size=4, accumulation_steps=4, lr=2e-5):
    accelerator = Accelerator(mixed_precision="bf16")
    global tokenizer,model
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token
    model = GPT2LMHeadModel.from_pretrained("gpt2", use_cache=False)
    model.to(accelerator.device)

    # Prepare datasets and dataloaders
    train_dataloader, validation_dataloader = prepare_dataloader(train_data, val_data, tokenizer, batch_size)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    # Prepare with accelerator
    model, optimizer, train_dataloader, validation_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader, validation_dataloader
    )

    # Attempt to load from the last checkpoint
    start_epoch = load_checkpoint(accelerator)

    # # Training loop
    # for epoch in range(start_epoch, epochs):
    #     model.train()
    #     total_loss, correct_predictions, total_predictions = 0, 0, 0
        
    #     with tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs}", disable=not accelerator.is_local_main_process) as progress_bar:
    #         for i, batch in enumerate(progress_bar):
    #             context = batch['input_ids'].to(accelerator.device, non_blocking=True)
    #             label = batch['labels'].to(accelerator.device, non_blocking=True)
    #             loss_mask = batch['loss_mask'].to(accelerator.device, non_blocking=True)

    #             with accelerator.autocast():
    #                 outputs = model(context)
    #                 logits = outputs.logits[:, :label.size(1), :]
    #                 masked_loss = torch.nan_to_num(masked_loss_fn(logits, label, loss_mask))

    #             accelerator.backward(masked_loss / accumulation_steps)
                
    #             if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_dataloader):
    #                 optimizer.step()
    #                 optimizer.zero_grad()

    #             total_loss += masked_loss.item() / accumulation_steps
    #             predictions = torch.argmax(logits, dim=-1)
    #             correct_predictions += ((predictions == label) * loss_mask).sum().item()
    #             total_predictions += loss_mask.sum().item()
                
    #             # Update live accuracy and loss in progress bar
    #             accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0
    #             progress_bar.set_postfix(loss=total_loss / (i + 1), accuracy=accuracy)

    #     avg_train_loss = total_loss / len(train_dataloader)
    #     train_accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0
    #     print(f"Epoch {epoch + 1} Training Loss: {avg_train_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

    #     # Save checkpoint at the end of each epoch
    #     save_checkpoint(model, optimizer, accelerator, epoch)
        
    # Evaluate on validation set
    val_loss, val_accuracy, bleu_score, rouge_score, perplexity = evaluate_model(model, validation_dataloader, accelerator, tokenizer)
    # Extract the main BLEU and ROUGE scores from dictionaries
    # bleu_main_score = bleu_score['bleu']
    rouge1 = rouge_score['rouge1']
    rouge2 = rouge_score['rouge2']
    rougeL = rouge_score['rougeL']
    
    # Print formatted output for the main scores
    print(f"Epoch Validation Loss: {val_loss:.4f}, "
        f"Validation Accuracy: {val_accuracy:.2f}%, "
        #   f"BLEU Score: {bleu_main_score:.4f}, "
        f"ROUGE-1: {rouge1:.4f}, "
        f"ROUGE-2: {rouge2:.4f}, "
        f"ROUGE-L: {rougeL:.4f}, "
        f"Perplexity: {perplexity:.4f}")
    print("bleu score ",bleu_score)
        

    # # Final model save
    # accelerator.save(model.state_dict(), MODEL_SAVE_PATH)
    # print("Training completed, and model saved at final location.")
    # Return the model and tokenizer for evaluation
    return model, tokenizer

In [11]:
from accelerate import notebook_launcher
# Run the training pipeline (adjust arguments as needed)
notebook_launcher(training_pipeline, args=(train_gpt_data, validation_gpt_data,EPOCHS,BATCH_SIZE), num_processes=1)

Launching training on one GPU.


c:\Users\Ikshvaku Rastogi\Documents\Documents\Minor Project\New folder\temp2\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Checkpoint loaded from ./checkpoints2\last_checkpoint
epoch_loaded


Epoch Validation Loss: 1.1254, Validation Accuracy: 76.92%, ROUGE-1: 0.8287, ROUGE-2: 0.6480, ROUGE-L: 0.7433, Perplexity: 3.0814
bleu score  {'bleu': 0.648246196430173, 'precisions': [0.837145829290844, 0.6611913224446131, 0.5885792274720839, 0.5485752122691898], 'brevity_penalty': 0.9970056830457246, 'length_ratio': 0.9970101570771565, 'translation_length': 5395809, 'reference_length': 5411990}


In [18]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [45]:

train_dataloader, validation_dataloader = prepare_dataloader(train_gpt_data, validation_gpt_data, tokenizer, 5)

In [67]:
from IPython.display import FileLink
FileLink(r'checkpoints2.zip')

/kaggle/working/checkpoints2.zip

In [66]:
import shutil
shutil.make_archive('checkpoints2','zip','checkpoints')

'/kaggle/working/checkpoints2.zip'

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel, GPT2Tokenizer, GPT2LMHeadModel
import faiss
from transformers import AutoModelForSequenceClassification,BertTokenizer
# Load BioBERT tokenizer and model
biobert_tokenizer = BertTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
biobert_model = AutoModelForSequenceClassification.from_pretrained(
    "dmis-lab/biobert-base-cased-v1.2",
    num_labels=1
)

biobert_model.eval()  # Set model to evaluation mode

# Load BioBERT weights from checkpoint
try:
    checkpoint_path = "./biobert/checkpoint.pth"
    checkpoint = torch.load(checkpoint_path)
    biobert_model.load_state_dict(checkpoint['model_state_dict'])
    print("BioBERT model weights loaded successfully.")
except Exception as e:
    print(f"Error loading BioBERT checkpoint: {e}")

# Load GPT-2 tokenizer and model
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token  # Set pad token to eos_token

gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")  # Load base GPT-2 model
gpt2_model.eval()  # Set model to evaluation mode
# Ensure the model is on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
biobert_model = biobert_model.to(device)
gpt2_model = gpt2_model.to(device)

# Load GPT-2 weights from checkpoint
try:
    gpt2_checkpoint_path = "./finalmodel2.pth"
    gpt2_checkpoint = torch.load(gpt2_checkpoint_path)
    gpt2_model.load_state_dict(gpt2_checkpoint)
    print("GPT-2 model weights loaded successfully.")
except Exception as e:
    print(f"Error loading GPT-2 checkpoint: {e}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Ikshvaku Rastogi\AppData\Local\Temp\ipykernel_9184\1292321941.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by th

BioBERT model weights loaded successfully.


c:\Users\Ikshvaku Rastogi\Documents\Documents\Minor Project\New folder\temp2\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\Ikshvaku Rastogi\AppData\Local\Temp\ipykernel_9184\1292321941.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be execu

GPT-2 model weights loaded successfully.


In [2]:
answer_index = faiss.read_index('./answer_index.faiss')

In [3]:
import pandas as pd
import numpy as np
train_gpt_data = pd.read_pickle('./train_gpt_input_data.pkl')
validation_gpt_data = pd.read_pickle('./validation_gpt_input_data.pkl')

In [4]:
def prepare_gpt_input(question, question_embedding, tokenizer , top_k=20, similarity_threshold=0.8):
    # Normalize and search FAISS index
    faiss.normalize_L2(np.array([question_embedding]).astype('float32'))
    distances, indices = answer_index.search(np.array([question_embedding]), top_k)
    
    # Filter based on similarity threshold
    similar_qas = [
        (train_gpt_data.iloc[idx]['question'], train_gpt_data.iloc[idx]['answer'])
        for idx, dist in zip(indices[0], distances[0])
        if dist >= similarity_threshold
    ]
    
    # Generate context string
    seen_questions = set()
    context_string = f"`QUESTION: {question} `ANSWER:"

    # Limit number of questions in context
    max_questions_in_context = 5  # Adjust this number based on how much context you want to provide

    for q, a in similar_qas:
        if q not in seen_questions:
            pair = f"`QUESTION: {q} `ANSWER: {a} "
            pair = pair.replace("\n","")
            if len(tokenizer.encode(pair + context_string)) < 1024:
                context_string = pair + context_string
                seen_questions.add(q)
            if len(seen_questions) >= max_questions_in_context:
                break  # Stop adding more questions after max_questions_in_context

    # Tokenize and truncate
    tokenized_input = tokenizer.encode(context_string, truncation=True, max_length=1024)
    return tokenized_input[-1024:]


In [5]:
import contractions,re
# Expanding contractions specifically for decontraction handling
def decontractions(phrase):
    phrase = contractions.fix(phrase).lower()  # Expand contractions
    phrase = re.sub(r"[^\w\s]", "", phrase)  # Remove punctuation
    return phrase

In [6]:
# Function to prepare inputs and extract embeddings
def extract_embeddings(text, model, tokenizer, max_length=512):
    # Tokenize the input text
    inputs = tokenizer(text, padding='max_length', max_length=max_length, truncation=True, return_tensors="pt")
    input_ids = inputs['input_ids'].to('cuda')
    attention_mask = inputs['attention_mask'].to('cuda')
    
    # Get embeddings from hidden states of the last layer
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        
        # Use the last hidden state as sentence embeddings, averaging across tokens
        last_hidden_state = outputs.hidden_states[-1]  # Get last layer
        embeddings = last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    
    return embeddings

In [17]:
def extract_and_combine_unique_answers(decoded_output):
    # Split the output by "QUESTION:" to separate the questions and answers
    parts = decoded_output.split("`QUESTION: ")
    
    # Initialize a set to track seen answers and a list to store the unique answers
    seen_answers = set()
    unique_answers = []

    # Iterate through each part starting from the second element (first question part)
    for part in parts[1:]:
        # Extract answer by splitting on "ANSWER:"
        if "`ANSWER:" in part:
            answer_start = part.split("`ANSWER: ")[1].strip()  # Get answer text
           
            if answer_start not in seen_answers:
                seen_answers.add(answer_start)
                unique_answers.append(answer_start)
    
    # Combine all unique answers
    return " ".join(unique_answers)



In [11]:


# Generate answer using the integrated pipeline
def generate_answer(question, answer_length=50):
    # Preprocess and truncate the input question
    preprocessed_question = decontractions(question)
    

    # Generate BioBERT embeddings
    
    question_embedding = extract_embeddings(preprocessed_question,biobert_model,biobert_tokenizer)
    
    # Prepare GPT input using FAISS results
    gpt_input = prepare_gpt_input(preprocessed_question, question_embedding, gpt2_tokenizer)
    
    # Ensure GPT input does not exceed length limit
    if len(gpt_input) > (1024 - answer_length):
        gpt_input = gpt_input[-(1024 - answer_length):]
    
    # Generate answer with GPT-2
    input_ids = torch.tensor([gpt_input]).to(device)
    gpt_output = gpt2_model.generate(input_ids=input_ids, max_length=1024, temperature=0.9, top_k=50, top_p=0.9)
    decoded_output = gpt2_tokenizer.decode(gpt_output[0])
    
    # # Extract and return the generated answer
    # answer_start = decoded_output.rindex("`ANSWER: ") + len("`ANSWER: ")
    return decoded_output

# Final function for end-to-end pipeline
def final_pipeline(question):
    # Generate the answer using the GPT-2 model
    generated_answer = generate_answer(question)

    return extract_and_combine_unique_answers(generated_answer)
    
    return generated_answer



In [12]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [19]:
final_pipeline("what will fix a headache and fever for a patient on blood pressure medicine")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'energy drinks is just one reason for these symptoms anxiety and heart disease are the major reasons whatever be the because meditation and yoga therapy will help settle down the heart rate it can some of the muscles of the neck attach directly to the skull this can cause a muscle tension type headache you are condition can cause you chest pain yes but regarding the pain related to eating where exactly is there any other symptoms respiratory or gastric troubles  back braces will help alot fever is not dangerous it is biologically more protective and therapeutic a lowerthannormal body temperature is often seen in viruses and is of no medical consequence uncontrolled high blood pressure can cause strokes heart failure blindness kidney failure and heart attacks most people with high blood pressure need to take medicines to treat it for their whole lives if the patient is taking a medication that is not helping they should seek medical attention immediately hi this is a knee replacement yo

In [ ]:
'`QUESTION: what are the symptoms of an adult brain tumor `ANSWER: a doctor should be seen if the following symptoms appear frequent headaches vomiting loss of appetite changes in mood and personality changes in ability to think and learn seizures `QUESTION: what are the symptoms of gastroenteritis `ANSWER: does crackers help with nausea `QUESTION: can stress because fevers in toddlers `ANSWER: stress is not considered a because of fever `QUESTION: would a sprained neck induce a headache `ANSWER: it can some of the muscles of the neck attach directly to the skull this can cause a muscle tension type headache `QUESTION: can pain because a fever 49yo male croniccopd gerds cronic pancreatitis arthritis bursitis on a long acting opiot pain reliever drove 325 miles to see dr got home and had 102 fever `ANSWER: no but what is causing the fever could be the because of the pain `QUESTION: what will fix a headache and fever  `ANSWER: a headache and fever are not related symptoms of a migraine `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a migraine nerve pathway pain and swelling `QUESTION: what is a migraine `ANSWER: a migraine is a type of headache that is caused by a'

'`QUESTION: would a sprained neck induce a headache `ANSWER: it can some of the muscles of the neck attach directly to the skull this can cause a muscle tension type headache `QUESTION: can pain because a fever 49yo male croniccopd gerds cronic pancreatitis arthritis bursitis on a long acting opiot pain reliever drove 325 miles to see dr got home and had 102 fever `ANSWER: no but what is causing the fever could be the because of the pain `QUESTION: what will fix a headache and fever  `ANSWER ? a headache and fever are symptoms of a migraine and a headache and fever are signs of a migraine and a headache and fever `QUESTION: what is a migraine `ANSWER: a migraine is a feeling of intense intense pain that is accompanied by a headache `QUESTION: what is a migraine `ANSWER: a migraine is a feeling of intense intense pain that is accompanied by a headache `QUESTION: what is a migraine `ANSWER: a migraine is a feeling of intense intense pain that is accompanied by a headache `QUESTION: what 

In [ ]:
'energy drinks is just one reason for these symptoms anxiety and heart disease are the major reasons whatever be the because meditation and yoga therapy will help settle down the heart rate it can some of the muscles of the neck attach directly to the skull this can cause a muscle tension type headache you are condition can cause you chest pain yes but regarding the pain related to eating where exactly is there any other symptoms respiratory or gastric troubles  back braces will help alot fever is not dangerous it is biologically more protective and therapeutic a lowerthannormal body temperature is often seen in viruses and is of no medical consequence uncontrolled high blood pressure can cause strokes heart failure blindness kidney failure and heart attacks most people with high blood pressure need to take medicines to treat it for their whole lives if the patient is taking a medication that is not helping they should seek medical attention immediately hi this is a knee replacement you need to do a mri on the affected area and a ct scan on the affected area as soon as possible good luck'

In [82]:
train_gpt_data.sample(5)['question'].values

array(['i started having pain and burning while urinating a week back',
       'is it okay to ride horses while pregnant  i have ridden for years and it is not like i would ride crazy horses or go faster than a walk i know what i am doing and do not see the harm in a slow walk',
       'what are the most healthy foods to stock in my refrigerator and freezer',
       'how to recover from erectile dysfunction caused by tobacco use  i smoked or chewed my whole adult life there has also been some alcohol abuse as well i am currently 45 and starting 6 months after a vasectomy i had when i was 36 is when i started having problems maintaining and erection for very long and between whatever is wrong with me and the psychological stress of wanting sex but knowing i cannot perform i have quit smoking drinking and using tobacco altogether i have periodontal disease the bone loss is severe i have experimented with viagra fail',
       'can i take a anti depressant with my adderall  i have been tak